<a href="https://colab.research.google.com/github/LeylaHu820085/-Coursera_Capstone/blob/main/Copia_de_Proyecto_final_semana_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clasificar a los pobladores de la ciudad de Toronto según sus características socioeconomicas, habitos, preferencias y lugares mas concurridos con ayuda de foursquare

Esto facilitará la decicision de la ubicación de la nueva pastelería de acuerdo a los lugares más transitados de  uestro público objetivo.

Introducción:
La informaciónd e fourquare permite realizar una segmentación de los potenciales clientes y poder tomar decisiones para ubicar un nuevo negocio orientado a dicho segmento.

Descripción del problema y discusión de antecedentes Encontrar una ubicación oportuna para ubicar una panadería y pastelería en la ciudad de Toronto. Toronto es una ciudad con mucha actividad económica y gran cantidad de población. Por tanto es importante conocer las preferencias de los ciudadanos según su estilo de vida, preferencias, hábitos y actividades que con frecuencia realizan, así como sus rutinas y los lugares que más frecuentan, todo ello para poder proponer una carta acorde al segmento objetivo dentro de la ciudad. La ubicación debe ser cercana a los lugares que más frecuenta el publico objetivo de la panadería y pastelería acorde al nivel socioeconómico de los potenciales clientes.

Descripción de los datos y cómo se utilizarán para resolver el problema. Los datos que se necesitará para desarrollar esta investigación son del CENSO ya que nos brindará información demógrafica, social y económica de las personas en la ciudad de Toronto Se usará Foursquare para obtener la información sobre los lugares frecuentados.

Datos Los datos que se necesitará para desarrollar esta investigación son del CENSO ya que nos brindará información demográfica, social y económica de las personas en la ciudad de Toronto. La data de la población es del año 2008 Enlace: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M Para lo cual se necesitará foursquare para obtener la información de las cafeterias de Toronto Link: https://foursquare.com/explore?mode=url&ne=44.418088%2C-78.362732&q=Restaurant&sw=42.742978%2C-80.554504

In [ ]:
import pandas as pd
import numpy as np

# ¡Hola, Proyecto Final del Curso!

In [ ]:
#importacion de librerias

import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


In [ ]:
#Importamos las tablas de la pagina de Wikipedia con el uso de la librería pandas
toronto= pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788')
toronto

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [ ]:
df_code_toronto=toronto[0]
df_code_toronto.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
#Se borra los municipios no asignados
df_code_toronto = df_code_toronto[df_code_toronto['Borough']!= 'Not assigned']
df_code_toronto = df_code_toronto.reset_index(drop=True)
df_code_toronto.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
#Se determina si hay vecindarios no asisgnados
df_code_toronto[df_code_toronto['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [ ]:
# Tamaño del dataframe.
print('El dataframe tiene {} filas'.format(df_code_toronto.shape[0]), 'y {} columnas'.format(df_code_toronto.shape[1]))
# Guardamos el dataframe como un archivo csv
df_code_toronto.to_csv('df_toronto_1.csv', index=True)

El dataframe tiene 103 filas y 3 columnas


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
latitude_longitude= pd.read_csv('Geospatial_Coordinates.csv')
latitude_longitude.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
df_code_toronto = pd.merge(df_code_toronto, latitude_longitude, on='Postal Code')
df_code_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
df_code_toronto.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

In [ ]:
boroughs = df_code_toronto['Borough'].unique().tolist()

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [ ]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [ ]:
import folium

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

for lat, lng, borough, neighbourhood in zip(df_code_toronto['Latitude'], 
                                           df_code_toronto['Longitude'],
                                           df_code_toronto['Borough'], 
                                           df_code_toronto['Neighbourhood']):
    label_text = borough + ' - ' + neighbourhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.8).add_to(map_toronto)  
    
map_toronto